In [ ]:
import sagemaker
my_sagemaker_role = sagemaker.get_execution_role()

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

In [ ]:
!pip install --upgrade sagemaker

In [ ]:
import tarfile
import os
import shutil

def extract_and_inspect_model(tar_path, extract_dir="tmp_inspect"):
    # Clean up old temp dir
    shutil.rmtree(extract_dir, ignore_errors=True)
    os.makedirs(extract_dir, exist_ok=True)

    # Extract tar.gz
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(extract_dir)

    # Find model directory
    for root, dirs, files in os.walk(extract_dir):
        if "saved_model.pb" in files:
            model_path = root
            break
    else:
        raise FileNotFoundError("No saved_model.pb found in extracted archive.")

    print(f"✅ Found SavedModel at: {model_path}")
    
    # Run saved_model_cli
    os.system(f"saved_model_cli show --all --dir {model_path}")

In [ ]:
extract_and_inspect_model("YueFeng_Test/saved_model_United_States.tar.gz")

In [ ]:
import os
from sagemaker import Session

def upload_all_models_to_s3(source_dir="models_for_deploy", s3_prefix="model"):
    sess = Session()
    bucket = sess.default_bucket()
    s3_paths = {}

    for filename in os.listdir(source_dir):
        if filename.endswith(".model.tar.gz"):
            local_path = os.path.join(source_dir, filename)
            print(f"☁️ Uploading {filename} → s3://{bucket}/{s3_prefix}/{filename}")
            
            s3_uri = sess.upload_data(
                path=local_path,
                key_prefix=s3_prefix
            )
            s3_paths[filename] = s3_uri
            print(f"✅ Uploaded to: {s3_uri}")

    return s3_paths
    
uploaded_model_uris = upload_all_models_to_s3()

In [ ]:
tf_model = TensorFlowModel(
    model_data="s3://my-sagemaker-bucket/.../saved_model_United_States.tar.gz",
    role=my_sagemaker_role,
    framework_version="2.9",
    py_version="py3"           # Now recognized in newer SageMaker SDK
)

predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)
